In [1]:
import numpy as np

import trax
from trax import layers as tl
from trax.supervised import lr_schedules, training

## Linear Regression
## Generate data for linear model

In [2]:
def get_data_linear():
    while True:
        x = np.random.randint(low=1, high=10) * 1.0
        y = x * 2.0 - 1
        yield (np.array([x]), np.array([y]))

In [3]:
data_linear = get_data_linear()
print(next(data_linear))

(array([2.]), array([3.]))


In [4]:
data_pipeline = trax.data.Serial(trax.data.Batch(50), trax.data.AddLossWeights(),)
data_stream = data_pipeline(data_linear)

## Build a simple linear model

In [5]:
model_linear = tl.Serial(tl.Dense(1))

## Train a linear model

In [6]:
# Use the same data_stream for both training and evaluation
train_task = training.TrainTask(
    labeled_data=data_stream,
    loss_layer=tl.L2Loss(),
    optimizer=trax.optimizers.SGD(0.01),
    n_steps_per_checkpoint=10,
)

eval_task = training.EvalTask(
    labeled_data=data_stream, metrics=[tl.L2Loss()], n_eval_batches=10,
)

In [7]:
# Delete the training folder
!rm -r linear_model

In [8]:
training_loop = training.Loop(
    model_linear, train_task, eval_tasks=[eval_task], output_dir="./linear_model"
)
training_loop.run(200)


Step      1: Total number of trainable weights: 2
Step      1: Ran 1 train steps in 0.15 secs
Step      1: train L2Loss |  332.65350342
Step      1: eval  L2Loss |  66.64659119

Step     10: Ran 9 train steps in 0.03 secs
Step     10: train L2Loss |  8.88021851
Step     10: eval  L2Loss |  0.44479808

Step     20: Ran 10 train steps in 0.03 secs
Step     20: train L2Loss |  0.47209778
Step     20: eval  L2Loss |  0.40909106

Step     30: Ran 10 train steps in 0.03 secs
Step     30: train L2Loss |  0.42413217
Step     30: eval  L2Loss |  0.40334216

Step     40: Ran 10 train steps in 0.04 secs
Step     40: train L2Loss |  0.35656795
Step     40: eval  L2Loss |  0.37079361

Step     50: Ran 10 train steps in 0.04 secs
Step     50: train L2Loss |  0.35460237
Step     50: eval  L2Loss |  0.33738366

Step     60: Ran 10 train steps in 0.04 secs
Step     60: train L2Loss |  0.30924049
Step     60: eval  L2Loss |  0.33994192

Step     70: Ran 10 train steps in 0.03 secs
Step     70: train L2

## Make a prediction 

In [20]:
test_data=np.array([[2.0],[3.0],[10.0],[44.0]])
model_linear(test_data)

DeviceArray([[ 3.4674654],
             [ 5.358649 ],
             [18.596935 ],
             [82.89718  ]], dtype=float32)

## Logistic Regression
## Generate data for logistic model

In [9]:
def get_data_logistic():
    while True:
        x = np.random.randint(low=-10, high=10) * 1.0

        if x < 0:
            y = 0
        else:
            y = 1

        yield (np.array([x]), np.array([y]))

In [10]:
data_logistic = get_data_logistic()
next(data_logistic)

(array([4.]), array([1]))

In [11]:
data_pipeline = trax.data.Serial(trax.data.Batch(50), trax.data.AddLossWeights(),)
data_stream = data_pipeline(data_logistic)

## Build a simple logistic model

In [12]:
model_logistic = tl.Serial(tl.Dense(1), tl.Sigmoid())

## Train a logistic model

In [13]:
train_task = training.TrainTask(
    labeled_data=data_stream,
    loss_layer=tl.BinaryCrossEntropyLoss(),
    optimizer=trax.optimizers.Adam(),
    lr_schedule=lr_schedules.warmup_and_rsqrt_decay(5, 0.01),
    n_steps_per_checkpoint=10,
)

eval_task = training.EvalTask(
    labeled_data=data_stream,
    metrics=[tl.BinaryCrossEntropyLoss()],
    n_eval_batches=10,
)

In [14]:
# Delete the training folder
!rm -r logistic_model

In [15]:
training_loop = training.Loop(
    model_logistic, train_task, eval_tasks=[eval_task], output_dir="./logistic_model"
)

In [16]:
training_loop.run(200)


Step      1: Total number of trainable weights: 2
Step      1: Ran 1 train steps in 0.39 secs
Step      1: train BinaryCrossEntropyLoss |  6.38921833
Step      1: eval  BinaryCrossEntropyLoss |  5.99964046

Step     10: Ran 9 train steps in 0.03 secs
Step     10: train BinaryCrossEntropyLoss |  6.16716242
Step     10: eval  BinaryCrossEntropyLoss |  5.76738453

Step     20: Ran 10 train steps in 0.03 secs
Step     20: train BinaryCrossEntropyLoss |  5.61083984
Step     20: eval  BinaryCrossEntropyLoss |  5.51198244

Step     30: Ran 10 train steps in 0.04 secs
Step     30: train BinaryCrossEntropyLoss |  5.33438587
Step     30: eval  BinaryCrossEntropyLoss |  5.29338312

Step     40: Ran 10 train steps in 0.03 secs
Step     40: train BinaryCrossEntropyLoss |  5.15196133
Step     40: eval  BinaryCrossEntropyLoss |  5.10557127

Step     50: Ran 10 train steps in 0.03 secs
Step     50: train BinaryCrossEntropyLoss |  4.78580523
Step     50: eval  BinaryCrossEntropyLoss |  4.90630245

Ste

## Display using Tensorboard
Tensorboard just needs to point to the training folder.

In [17]:
# First load the extension
%load_ext tensorboard

In [18]:
# For a linear model,
%tensorboard --logdir linear_model